In [ ]:
!pip install cellpose

In [1]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 200
import random
import PIL

In [ ]:
analysis_dir = '/path/to/dir'
data_dir = '/path/to/data'

In [4]:
import pandas as pd
df = pd.read_csv(analysis_dir + '/data.csv')

In [5]:
# Filter just the Organoid Group normal
df = df[df['Organoid Group'] == 'Tumor']
control_df = df[df['Control / Treatment?'] == 'Control']
treatment_df = df[df['Control / Treatment?'] == 'Treatment']

In [6]:
all_files = []
import os
for root, dirs, files in os.walk(data_dir):
  for fn in files:
    if fn.endswith(".tif"):
      all_files.append(os.path.join(root, fn))


In [8]:
file2path = {x.split('/')[-1]:x for x in all_files}

In [9]:
DAPI = '_DAPI'
PHAL = '_PHAL'
ECAD = '_ECAD'
PHAL_DAPI = '_PHAL+DAPI'
ECAD_DAPI = '_ECAD+DAPI'
def get_image(name, suffix):
  filename = name+suffix
  return file2path.get(filename+'.tif')

In [10]:
import pickle

with open(analysis_dir + '/control_masks.pkl', 'rb') as f:
  control_masks = pickle.load(f)
with open(analysis_dir + '/treatment_masks.pkl', 'rb') as f:
  treatment_masks = pickle.load(f)

EXCLUDED_NAMES = []

In [17]:
import pandas as pd
from cellpose import transforms
from scipy import ndimage
from skimage import measure
from tqdm import tqdm
import cv2

def resize_image(fn, size=400):
  img = PIL.Image.open(fn)
  img = np.array(img)
  return transforms.resize_image(img, Lx=size, Ly=size)

def get_nucleus_mask(dapi):
  _, mask = cv2.threshold(dapi, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  return mask

def get_features_dict(name, mask, size_thresh=30):
  dapi = resize_image(get_image(name, DAPI))[:,:,2] # B
  ph = resize_image(get_image(name, PHAL))[:,:,1] # G
  ecad = resize_image(get_image(name, ECAD))[:,:,0] # R

  labels = np.unique(mask)

  # Remove background (usually label 0)
  labels = labels[labels != 0]

  results = []
  for label in labels:
    # Create a binary mask for this label
    binary_mask = (mask == label)

    # Find bounding box
    slice_y, slice_x = ndimage.find_objects(binary_mask)[0]

    # Extract the patch
    patch_mask = binary_mask[slice_y, slice_x]

    dapi_patch = dapi[slice_y, slice_x]
    ph_patch = ph[slice_y, slice_x]
    ecad_patch = ecad[slice_y, slice_x]
    dapi_pixels = dapi_patch[patch_mask]
    ph_pixels = ph_patch[patch_mask]
    ecad_pixels = ecad_patch[patch_mask]

    nucleus_mask = patch_mask & get_nucleus_mask(dapi_patch)
    ph_on_nucleus = ph_patch[nucleus_mask].mean()
    ecad_on_nucleus = ecad_patch[nucleus_mask].mean()
    dapi_on_nucleus = dapi_patch[nucleus_mask].mean()

    ph_ecad_corr = np.corrcoef(ph_pixels, ecad_pixels)[0,1]

    if np.sum(mask) < size_thresh:
      continue

    res =  {
        'name': name,
        'label': label,
        'area': np.sum(mask),
        'bbox': (slice_y.start, slice_x.start, slice_y.stop, slice_x.stop),
        'dapi_mean': np.mean(dapi_pixels),
        'dapi_std': np.std(dapi_pixels),
        'ph_mean': np.mean(ph_pixels),
        'ph_std': np.std(ph_pixels),
        'nucleus_area': nucleus_mask.sum(),
        'ecad_mean': np.mean(ecad_pixels),
        'ecad_std': np.std(ecad_pixels),
        'ph_on_nucleus': ph_on_nucleus,
        'ecad_on_nucleus': ecad_on_nucleus,
        'dapi_on_nucleus': dapi_on_nucleus,
        'ph_ecad_corr': ph_ecad_corr,
        'relative_ph_on_nucleus': ph_on_nucleus / np.mean(ph_pixels)
    }
    results.append(res)
  return results

  return [dict(name=name)]

def get_features(image_to_mask, thresh=60):
  features = []
  for name in tqdm(image_to_mask):
    if name in EXCLUDED_NAMES:
      continue
    mask = image_to_mask[name]
    candidates = get_features_dict(name, mask)
    features.extend(candidates)
  return pd.DataFrame(features)

In [ ]:
control_features = get_features(control_masks)
treatment_features = get_features(treatment_masks)

In [27]:
control_features.to_csv(analysis_dir + '/control_features.csv')
treatment_features.to_csv(analysis_dir + '/treatment_features.csv')

In [ ]:
control_features.describe()

In [ ]:
treatment_features.describe()

In [ ]:
# prompt: # plot histograms of ecad_mean in the control and treatment, with some alpha. Add legend
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(6,4))
plt.hist(control_features['nucleus_area'], density=True, alpha=0.5, label='Control', bins=30)
plt.hist(treatment_features['nucleus_area'], density=True, alpha=0.5, label='Treatment', bins=30)
plt.xlabel('Nucleus area [arbitrary units]')
plt.title('Nucleus area')
plt.legend()

In [ ]:
# prompt: # plot histograms of ecad_mean in the control and treatment, with some alpha. Add legend
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(6,4))
plt.hist(control_features['ecad_mean'], alpha=0.5, label='Control', bins=30)
plt.hist(treatment_features['ecad_mean'], alpha=0.5, label='Treatment', bins=30)
plt.xlabel('ECAD Mean [arbitrary units]')
plt.title('ECAD MEAN')
plt.legend()